# Notebook Lab : Le réseau de neurones qui écrivait des romans

Contacts : 

*   c.hannotte@groupeonepoint.com
*   b.mathieu@groupeonepoint.com
          

# Initialisation

Installer et importer les bibliothèques nécessaires

In [1]:
!pip install huggingface_hub datasets tensorflow==2.11 transformers==4.26.0 nltk keras==2.11 flask flask-socketio futures==2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import keras
import nltk
import pandas as pd
import tensorflow as tf
import datasets
from datasets import Dataset, DatasetDict, Features, Value, load_dataset
from huggingface_hub import HfFolder
from transformers import pipeline
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig
from datasets import load_dataset, load_from_disk
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback
from huggingface_hub import HfFolder

from transformers import create_optimizer
from transformers import DataCollatorForLanguageModeling
from transformers import pipeline

from datetime import datetime

from pathlib import Path

In [3]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Préparation des données (Script 1_prepare_dataset_solution.py) 1/2 à lancer pour avoir le dataset en local

In [4]:
def download_files(data_dict: dict) -> list:

    texts_train = []
    
    for key_name, value_url in data_dict.items():
        # print(key_name, value_url)
        filepath = keras.utils.get_file(f"{key_name}.txt", origin=value_url)
        with open(filepath, encoding="utf-8") as file:
            text = file.readlines()
            # On peut aussi enlever les sauts de lignes au début avec
            # text = file.read().splitlines()
            # Soit on le fait plus tard dans le clean
            print("\n")
            print("Livre :", key_name)
            print("Les deux premières lignes du fichier : ", text[:2])
            print("Nombre de lignes dans le fichier : ", len(text))

            # print(text[0:20])
            # On remarque que les 50 premières lignes sont l'intro et la préface de Gutenberg
            # On va les retirer pour avoir le moins de bruit possible


            for row_end in range(-1, -len(text), -1):
                if 'END' in text[row_end] or 'FIN' in text[row_end]:
                    print(text[row_end], row_end)
                    break



            texts_train.extend(text[50: row_end])

            # On remarque aussi que les dernières lignes comportent aussi du bruit et en anglais
            # On retire environ les 200 dernières lignes pour éviter d'incorporer de l'anglais

            # extend est différent d'append. [a].append([b]) = [a,[b]]. [a].extend([b]) = [a,b].

    return texts_train


def split_text_to_list(text_list: list) -> list:
    print(f"nb rows: {len(text_list)}")
    # On retire les espaces à droite et les \n pour chaque ligne
    cleaned_list = [line.rstrip() for line in text_list]

    # On crée une chaine de caractères qui concatène chaque phrase et les sépare d'un espace
    list_as_str = " ".join(cleaned_list)
    list_as_str = list_as_str.replace('_', '')

    # On utilise nltk pour séparer notre chaine en phrase
    # Séparer par un "." n'est pas suffisant pour identifier une phrase
    # Une phrase peut se finir par "!", "?", "...", "etc."
    # On charge  le tokenizer qui sépare le texte en phrase
    tokenizer = nltk.data.load("tokenizers/punkt/PY3/french.pickle")

    # Et on l'applique sur notre liste
    splitted_list = tokenizer.tokenize(list_as_str)
    # print(splitted_list)

    # On enlève le texte vide (des phrases sans aucun mots)
    text_list = list(filter(None, splitted_list))
    # Pourcentage de texte enlevé
    percent = 1 - len(text_list) / len(splitted_list)
    print(f"\n Pourcentage de phrases None enlevées {round(percent*100,2)}")
    print("Nombre de phrases complètes", len(text_list))
    return text_list


def prepare_dataset(data_dict: dict) -> list:
    text_str = download_files(data_dict)
    prepared_text_list = split_text_to_list(text_str)
    return prepared_text_list


def compute_sentence_length(example) -> str:
    return {"sentence_length": len(example["text"].split())}


#Préparation des données (Script 1_prepare_dataset_solution.py) 2/2 à lancer pour avoir le dataset en local

In [5]:
LELIA_URL = "https://www.gutenberg.org/files/39738/39738-0.txt"
LA_PETITE_FADETTE_URL = "https://www.gutenberg.org/cache/epub/34204/pg34204.txt"
GABRIEL_URL = "https://www.gutenberg.org/cache/epub/13380/pg13380.txt"
LETTRE_VOYAGEUR_URL = "https://www.gutenberg.org/files/37989/37989-0.txt"
LA_MARQUISE_URL = "https://www.gutenberg.org/cache/epub/13025/pg13025.txt"
DAME_VERTES_URL = "https://www.gutenberg.org/cache/epub/69098/pg69098.txt"
MEUNIER_ANGIBAULT_URL = "https://www.gutenberg.org/cache/epub/13892/pg13892.txt"
COMPTESSE_RUDOLSTADT_URL = "https://www.gutenberg.org/files/17225/17225-0.txt"
INDIANA_URL = "https://www.gutenberg.org/files/63445/63445-0.txt"
HISTOIRE_DE_MA_VIE_1_URL = "https://www.gutenberg.org/cache/epub/39101/pg39101.txt"
HISTOIRE_DE_MA_VIE_3_URL = "https://www.gutenberg.org/files/42765/42765-0.txt"
MARE_AU_DIABLE_URL = "https://www.gutenberg.org/files/23582/23582-0.txt"
VALENTINE_URL = "https://www.gutenberg.org/files/17251/17251-0.txt"


# Première étape créer un dictionnaire
# Clé du dictionnaire = Titre en chaine de caractères
# Valeur : Url (string) du fichier .txt à télécharger

dict_train = {
    "Lélia": LELIA_URL,
    "La petite fadette": LA_PETITE_FADETTE_URL,
    "Gabriel": GABRIEL_URL,
    "Lettre d'un voyageur": LETTRE_VOYAGEUR_URL,
    "La Marquise": LA_MARQUISE_URL,
    "Les dames vertes": DAME_VERTES_URL,
    "Le meunier d'Angibault": MEUNIER_ANGIBAULT_URL,
    "La comptesse de Rudolstadt": COMPTESSE_RUDOLSTADT_URL,
    'Indiana': INDIANA_URL,
    'Histoire de ma vie, livre 1': HISTOIRE_DE_MA_VIE_1_URL,
    'Histoire de ma vie, livre 3': HISTOIRE_DE_MA_VIE_3_URL,
}

dict_test = {"La Mare au Diable": MARE_AU_DIABLE_URL}

# On appelle nos fonctions qui nous renvoient une liste de phrases préparées

text_list_train = prepare_dataset(dict_train)

# On fait pareil pour le test

text_list_test = prepare_dataset(dict_test)

# Maintenant on est revenu au début, il mais nous avons un texte nettoyé.
# Il nous suffit de charger les données en créant un objet de la classe Dataset
# Cette fois ci pas en utilisant load_dataset() qui charge depuis un fichier (texte, csv, parquet...)
# Mais en créant une instance de la classe dataset.
# Rapel : Nous avons une liste de texte. Plusieurs options de chargement

# Créer un tableau pandas d'une colonne du nom "train_text" et charger notre liste
# Chaque phrase = 1 ligne du tableau
# On utilise la fonction from_pandas() venant du nom de la bibliothèque de dataframe
dataset_train = Dataset.from_pandas(
    pd.DataFrame({"train_text": text_list_train}))

# Ou Créer un dictionnaire et utiliser la fonction from_dict(), la clé est le nom de la colonne

# On ajoute le type de nos données avec la classe Features de Hugging Face
# On indique que la colonne "text" est de type string
features = Features({"text": Value(dtype="string")})
dataset_train = Dataset.from_dict({"text": text_list_train}, features)

print("Taille du dataset d'entrainement ", dataset_train.shape)

# On fait pareil pour le dataset de test, le typage des données est le même
dataset_test = Dataset.from_dict({"text": text_list_test}, features)

dataset = DatasetDict({"train": dataset_train, "test": dataset_test})
print("Dataset final: \n ", dataset)

# On est revenus à l'état initial mais en ayant modifié nos données
# On peut adapter les modifications selon le niveau de traitement (répartition statistique, découpage spécial)

# Autres traitements possibles...
# On peut calculer la taille de chaque phrase
# On peut utiliser directement des fonctions map sur la classe Dataset pour effectuer un traitement direct

dataset_new_col_train = dataset["train"].map(compute_sentence_length)
print(
    "Nouvelle colonne ajoutée", dataset_new_col_train[0]
)  # Ajout d'une nouvelle colonne

# Certaines phrases ont trop peu de mots
print(
    "Type de phrases trop courtes dans le texte : ",
    dataset_new_col_train.sort("sentence_length")[:3],
)
# On peut les supprimer (par exemple, retirer les phrases qui ont une taille <= 3 mots)
# Cela nous permettra d'avoir des phrases générées plus cohérentes (et plus longues)

dataset_new_col_train = dataset_new_col_train.filter(
    lambda x: x["sentence_length"] > 3)
print("Nombre de phrases dans le train initial : ", dataset_train.num_rows)
print("Nombre de phrases dans le train filtré : ",
      dataset_new_col_train.num_rows)

# On fait pareil pour le dataset de test

dataset_new_col_test = (
    dataset["test"]
    .map(compute_sentence_length)
    .filter(lambda x: x["sentence_length"] > 3)
)

# ----------------------------- TRAIN/VALIDATION/TEST ------------------------------------------------

# On sépare maintenant notre dataset d'entrainement (train) en deux sets
# 1. Un d'entrainement pur (train) qui nous permettra d'apprendre sur la donnée fournie
# 2. Un de vérification (validation) sur lequel on évaluera notre modèle.
# Le dataset de validation nous donne un indice sur notre future performance en conditions réelles (données inconnues)

print("\n ----------- SPLIT DATASET ---------\n")

dataset_train_splitted = dataset_new_col_train.train_test_split(
    train_size=0.9, seed=42)
# Par défaut ce dataset s'appelle test, on le renomme par "validation"
dataset_train_splitted["validation"] = dataset_train_splitted.pop("test")

print(dataset_train_splitted)

dataset_train_splitted.save_to_disk("aurore/data/")

1002764/1002764 [==============================] - 0s 0us/step


Livre : Lélia
Les deux premières lignes du fichier :  ['\ufeffThe Project Gutenberg EBook of Lélia, by George Sand\n', '\n']
Nombre de lignes dans le fichier :  16447
SEND DONATIONS or determine the status of compliance for any
 -41
365831/365831 [==============================] - 0s 0us/step


Livre : La petite fadette
Les deux premières lignes du fichier :  ['\ufeffThe Project Gutenberg EBook of La petite fadette, by George Sand\n', '\n']
Nombre de lignes dans le fichier :  6664
SEND DONATIONS or determine the status of compliance for any
 -41
265158/265158 [==============================] - 0s 0us/step


Livre : Gabriel
Les deux premières lignes du fichier :  ['\ufeffThe Project Gutenberg EBook of Gabriel, by George Sand\n', '\n']
Nombre de lignes dans le fichier :  7305
SEND DONATIONS or determine the status of compliance for any
 -41
737600/737600 [==============================] - 0s 0us/step


Livre : Lettre d'un v

  0%|          | 0/54957 [00:00<?, ?ex/s]

Nouvelle colonne ajoutée {'text': ' PREFACE ET NOTICE NOUVELLE  Prix: 1 franc 95 centimes  [Illustration]  PARIS MICHEL LÉVY FRÈRES, LIBRAIRES ÉDITEURS 2 BIS, RUE VIVIENNE, ET BOULEVARD DES ITALIENS, 15 A LA LIBRAIRIE NOUVELLE  1867  CALMANN LÉVY, ÉDITEUR 3.', 'sentence_length': 34}
Type de phrases trop courtes dans le texte :  {'text': ['To', '1.D.', 'Oh!'], 'sentence_length': [1, 1, 1]}


  0%|          | 0/55 [00:00<?, ?ba/s]

Nombre de phrases dans le train initial :  54957
Nombre de phrases dans le train filtré :  49228


...


 ----------- SPLIT DATASET ---------

DatasetDict({
    train: Dataset({
        features: ['text', 'sentence_length'],
        num_rows: 44305
    })
    validation: Dataset({
        features: ['text', 'sentence_length'],
        num_rows: 4923
    })
})


Flattening the indices:   0%|          | 0/45 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/44305 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/5 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/4923 [00:00<?, ? examples/s]

## Utilisation d'un RN (4_generate.py)

In [ ]:
MODEL_NAME  = 'benjamin/gpt2-wechsel-french'

#---------------- Chargement du Tokenizer et du modèle ----------------------------------

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

#TODO Construction de la configuration GPT2
config = AutoConfig.from_pretrained(
    #######,
    vocab_size=####,
    n_ctx=#####,
    bos_token_id=####,
    eos_token_id=####,
)

# TODO instancier le modèle

model = #### Chargement de la config dans le modèle
print("Construction du modèle")
model = #### appel de model et de from_pretrained pour charger le modèle


In [ ]:

#------------------- Création de la pipeline de génération ---------------------------------

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, device=0
)

#-------------------- Génération de texte --------------------------------------------------

# TODO :

prompts = # liste de phrase de prompt

output0= # output du premier élément de la liste prompts après appel à pipe()
output1= # output du second élément de la liste prompts après appel à pipe()

#### TODO Afficher les résultats




## Entraînement (3_train.py)

TODO : Afficher le summary du modèle

In [8]:
#------------------ Chargement des paramètres -----------
MODEL_NAME  = 'benjamin/gpt2-wechsel-french'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Configuration du réseau GPT2
config = AutoConfig.from_pretrained(
    MODEL_NAME,
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Initialisation du modele

model = TFGPT2LMHeadModel(config)
print("Construction du modèle")
model = model.from_pretrained(MODEL_NAME, from_pt=True)

model(model.dummy_inputs)

# TODO Appel de summary() sur model et l'afficher
summary = ###
print(summary)

SyntaxError: ignored

In [6]:
#------------------ Fonctions de tokenization du dataset -------------------------

def tokenize(element):
    print("Tokenization of the dataset.")
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=False,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


In [7]:
class Saver(Callback):
    VAL_LOSS = 'val_loss'

    def __init__(self, model) -> None:
        super().__init__()
        self.model = model
        self.best_val_loss = None

        now = datetime.now()
        date = now.strftime("%Y-%m-%d %H:%M:%S")

        self.model_path = MODEL_PATH / date

    def on_epoch_end(self, epoch, logs=None):
        if self.best_val_loss is None:
            self.best_val_loss = logs[Saver.VAL_LOSS]
            self.model.save_pretrained(self.model_path)
            logging.warning(f"\nInitialize saved model at epoch {epoch}\n")
        elif self.best_val_loss > logs[Saver.VAL_LOSS]:
            self.model.save_pretrained(self.model_path)
            logging.warning(f"\nUpdated saved model at epoch {epoch} (previous loss: {self.best_val_loss}, current loss: {logs[Saver.VAL_LOSS]}\n")
            self.best_val_loss = logs[Saver.VAL_LOSS]


TODO Charger le jeu de données

In [ ]:
#--------------------- Récupération du dataset et du tokenizer -----------------

# TODO : charger le jeu de données (local suite au lancement du 1_solution)
dataset = None
if not dataset:
    raise NotImplementedError

#-------------------------- Tokénisation du dataset de phrases ------------------

# Appel de la fonction tokenize qui va transformer chaque phrase du dataset en une phrase de token

tokenized_datasets =  dataset.map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)

print("Dataset tokenisé :", tokenized_datasets)


In [ ]:
#--------------------- Préparation des lots (batches) pour l'entrainement  ---------

tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

out = data_collator([tokenized_datasets["train"][i] for i in range(5)])

# for key in out:
#     print(f"{key} shape: {out[key].shape}")

# for key in out:
#     print(f"{key}: {out[key][0]}")
    

#--------------------------- ETAPE 3 : Dataset -> TF DATASET --------------------------

print("\n Conversion du dataset tokenisé en dataset tensorflow \n")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)
tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)

# ------------------------ Configuration du réseau --------------------------------------


num_train_steps = len(tf_train_dataset)

# lr scheduler pour améliorer la stabilité du réseau
# Au début lr élevé pour trouver l'optimum pour ensuite se stabiliser

optimizer, schedule = create_optimizer(
    init_lr=5e-4,
    num_warmup_steps=3000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

# Compilation du modèle

model.compile(optimizer=optimizer)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

TODO : Appel de la fonction fit() sur le modèle

In [ ]:
print("Dataset inital :", dataset)

tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)

print("Dataset tokenisé :", tokenized_datasets)


In [ ]:
# #---------------------------------- ENTRAINEMENT ---------------------------------------------------------------------
MODEL_PATH = Path('aurore')/'model'
print("\n Entrainement du modèle en cours ... \n")

# epochs = Nombre d'itérations. Attention à ne pas faire exploser votre machine :D
saver = Saver(model, tokenizer)


model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=1000, callbacks=[saver])



In [ ]:
print("\n Entrainement du modèle en cours ... \n")

#TODO : appeler la fonction fit sur le modele avec les datasets de validation et de train tensoflow,
# le nombre d'epochs et l'agument callbacks=[saver]
saver = Saver(model)
model.#########

print("Fin de l'entrainement, modèle sauvegardé en local ")
model.save_pretrained("aurore/model/")

# Tokenisation (2_tokenise.py)

TODO : Télécharger le tokenizer pré-entrainé via MODEL_NAME

In [ ]:
print("------------------ TOKENIZER PRE ENTRAINE -------------------")
# On donne une taille max de phrase possible
CONTEXT_LENGTH = 100

# On récupère un tokenizer pré entrainé sur du français pour gpt2 (sur hugging face il y en a pleins)

# TODO : Télécharger le tokenizer pré-entrainé via from_pretrained de AutoTokenizer
pretrained_tokenizer = None

if not pretrained_tokenizer:
    raise NotImplementedError

TODO : Créer une variable contenant un texte simple

In [ ]:
# TODO : Créer une variable contenant un texte simple

txt = ""

if not txt:
    raise NotImplementedError

print(f"Le vocabulaire a une taille de {len(pretrained_tokenizer)}")

TODO : Analyser le texte avec le tokenizer

In [ ]:
# TODO : Analyser ce texte avec le tokenizer 
tokens_analysis = {}

if not tokens_analysis:
    raise NotImplementedError

# Afficher les tokens
tokens = tokens_analysis['input_ids']

TODO : Utiliser convert_ids_to_tokens()

In [ ]:
# On remarque les symboles spéciaux Ġ et Ċ qui indiquent les espaces et les retours à la ligne.
print("Nombre associé à chaque token : \n",tokens)

# On peut reconvertir le string tokenisé en chaine de caractères et voir son découpage

# TODO : Utiliser convert_ids_to_tokens() pour afficher le texte associé à chaque token

converted = ''

if not converted:
    raise NotImplementedError

print("Chaine de caractères convertie en token : \n", converted, "\n")


Facultatif : Spécialiser un tokenizer

In [ ]:
# --------------------------- CAS 2 : Tokenizer pré-entrainé ---------------------------

# Créons un générator pour éviter que Python sauvegarde tout mémoire jusqu'au moment nécessaire

def get_training_corpus():
    batch_size = 1000
    return (
        downloaded_dataset["train"][i : i + batch_size]["text"]
        for i in range(0, len(downloaded_dataset["train"]), batch_size)
    )

# Todo appeler la fonction
training_corpus = ####
if not training_corpus:
    raise NotImplementedError

print("------------------ TOKENIZER CUSTOMISE -------------------")

In [ ]:


print("Analyse des phrases pour l'entrainement du tokenizer :")

for i, text in enumerate(get_training_corpus()):
    print(f"Batch {i} : {len(text)} phrases d'entrainement.")

# TODO : Utiliser train_new_from_iterator sur le tokenizer pré entrainé 
# Avec comme arguments le corpus d'entrainement et une taille de vocabulaire de 52000
vocab_size = None
tokenizer = None

if not tokenizer:
    raise NotImplementedError

print("Le vocabulaire a une taille de ", tokenizer.vocab_size)

txt = "Bonjour Madame, je m'appelle Georges Sand. Et vous ?"
tokens = tokenizer(txt)['input_ids']
# On remarque les symboles spéciaux Ġ et Ċ qui indiquent les espaces et les retours à la ligne.
print("Nombre associé à chaque token : \n",tokens)

# On peut reconvertir le string tokenisé en chaine de caractères et voir son découpage
converted = tokenizer.convert_ids_to_tokens(tokens)
print("Chaine de caractères convertie en token : \n", converted)


# Lequel est le meilleur ?


print(f"Il y a {len(tokenizer.tokenize(txt))} tokens pour le tokenizer customisé")
print(f"Il y a {len(pretrained_tokenizer.tokenize(txt))} tokens pour le tokenizer pré-entrainé")

# Le tokeniser qui sait le mieux généraliser ou celui qui permet d'avoir un token par mot ?


TODO : Sauvegarde du tokenizer en local

In [ ]:
path="aurore/"
file_name="tokenizer"

# TODO Sauvegarde du tokenizer en local avec save_pretrained
tokenizer.######

# Preparation du dataset (1_prepare_dataset.py)

In [ ]:
# ----------------------------- Chargement des données --------------------------------

LELIA_URL = "https://www.gutenberg.org/files/39738/39738-0.txt"
LA_PETITE_FADETTE_URL = "https://www.gutenberg.org/cache/epub/34204/pg34204.txt"
GABRIEL_URL = "https://www.gutenberg.org/cache/epub/13380/pg13380.txt"
LETTRE_VOYAGEUR_URL = "https://www.gutenberg.org/files/37989/37989-0.txt"
LA_MARQUISE_URL = "https://www.gutenberg.org/cache/epub/13025/pg13025.txt"
DAME_VERTES_URL = "https://www.gutenberg.org/cache/epub/69098/pg69098.txt"
MEUNIER_ANGIBAULT_URL = "https://www.gutenberg.org/cache/epub/13892/pg13892.txt"
COMPTESSE_RUDOLSTADT_URL = "https://www.gutenberg.org/files/17225/17225-0.txt"

MARE_AU_DIABLE_URL = "https://www.gutenberg.org/files/23582/23582-0.txt"

# Première étape créer un dictionnaire
# Clé du dictionnaire = Titre en chaine de caractères
# Valeur : Url (string) du fichier .txt à télécharger

dict_train = {
    "Lélia": LELIA_URL,
    "La petite fadette": LA_PETITE_FADETTE_URL,
    "Gabriel": GABRIEL_URL,
    "Lettre d'un voyageur": LETTRE_VOYAGEUR_URL,
    "La Marquise" : LA_MARQUISE_URL,
    "Les dames vertes" : DAME_VERTES_URL,
    "Le meunier d'Angibault" : MEUNIER_ANGIBAULT_URL,
    "La comptesse de Rudolstadt" : COMPTESSE_RUDOLSTADT_URL
}

dict_test = {"La Mare au Diable": MARE_AU_DIABLE_URL}


In [ ]:
# -------------------- Fonctions de data préparation --------------------------------


def download_files(data_dict: dict) -> list:

    for key_name, value_url in data_dict.items():

        print(key_name, value_url)

        # TODO : Utilise la fonction get_file

        filepath = ""

        if not filepath:
            raise NotImplementedError

        texts_train = []

        # TODO : Ouvrir le fichier téléchargé (filepath) avec la fonction with open

        # TODO : Récupérer les lignes du fichier avec readlines()

        # TODO : Retirer les 50 premières lignes (et les 100 dernières...)

        # Astuce, on peut utiliser extend sur une liste.
        # extend est différent d'append. [a].append([b]) = [a,[b]]. [a].extend([b]) = [a,b].

    if not texts_train:
        raise NotImplementedError

    return texts_train



In [ ]:
def split_text_to_list(text_list: list) -> list:

    # TODO : Modifier cleaned list pour retirer les espages et \n en fin de ligne
    cleaned_list = []

    if not cleaned_list:
        raise NotImplementedError

    # TODO : Re-créer le texte original avec la méthode join
    list_as_str = ""

    if not list_as_str:
        raise NotImplementedError

    # On utilise nltk pour séparer notre chaine en phrase
    # Séparer par un "." n'est pas suffisant pour identifier une phrase
    # Une phrase peut se finir par "!", "?", "...", "etc."
    # On charge  le tokenizer qui sépare le texte en phrase

    nltk_transformer = nltk.data.load("tokenizers/punkt/PY3/french.pickle")

    # TODO : Appeler NLTK pour tokenizer la chaine de caractères
    splitted_list = []

    if not splitted_list:
        raise NotImplementedError

    # print(splitted_list)

    # On enlève le texte vide (des phrases sans aucun mots)
    text_list = list(filter(None, splitted_list))
    # Pourcentage de texte enlevé
    percent = 1 - len(text_list) / len(splitted_list)
    print(f"\n Pourcentage de phrases None enlevées {round(percent*100,2)}")
    print("Nombre de phrases complètes", len(text_list))
    return text_list


In [ ]:
def prepare_dataset(data_dict: dict) -> list:
    text_str = download_files(data_dict)
    prepared_text_list = split_text_to_list(text_str)
    return prepared_text_list


def compute_sentence_length(example) -> str:
    return {"sentence_length": len(example["text"].split())}

In [ ]:
# On appelle nos fonctions qui nous renvoient une liste de phrases préparées

text_list_train = prepare_dataset(dict_train)

# On fait pareil pour le test

text_list_test = prepare_dataset(dict_test)

# Maintenant on est revenu au début, il mais nous avons un texte nettoyé.
# Il nous suffit de charger les données en créant un objet de la classe Dataset
# Cette fois ci pas en utilisant load_dataset() qui charge depuis un fichier (texte, csv, parquet)
# Mais en créant une instance de la classe dataset.
# Rapel : Nous avons une liste de texte. Plusieurs options de chargement

# Créer un tableau pandas d'une colonne du nom "train_text" et charger notre liste
# Chaque phrase = 1 ligne du tableau
# On utilise la fonction from_pandas() venant du nom de la bibliothèque de dataframe
dataset_train = Dataset.from_pandas(pd.DataFrame({"train_text": text_list_train}))

# Ou Créer un dictionnaire et utiliser la fonction from_dict(), la clé est le nom de la colonne

# On ajoute le type de nos données avec la classe Features de Hugging Face
# On indique que la colonne "text" est de type string
features = Features({"text": Value(dtype="string")})
dataset_train = Dataset.from_dict({"text": text_list_train}, features)

print("Taille du dataset d'entrainement ", dataset_train.shape)

# On fait pareil pour le dataset de test, le typage des données est le même
dataset_test = Dataset.from_dict({"text": text_list_test}, features)

dataset = DatasetDict({"train": dataset_train, "test": dataset_test})
print("Dataset final: \n ", dataset)

# On est revenus à l'état initial mais en ayant modifié nos données
# On peut adapter les modifications selon le niveau de traitement
# (répartition statistique, découpage spécial)

# Autres traitements possibles...
# On peut calculer la taille de chaque phrase
# On peut utiliser directement des fonctions map sur la classe Dataset
# pour effectuer un traitement direct

dataset_new_col_train = dataset["train"].map(compute_sentence_length)
print(
    "Nouvelle colonne ajoutée", dataset_new_col_train[0]
)  # Ajout d'une nouvelle colonne

# Certaines phrases ont trop peu de mots
print(
    "Type de phrases trop courtes dans le texte : ",
    dataset_new_col_train.sort("sentence_length")[:3],
)
# On peut les supprimer (par exemple, retirer les phrases qui ont une taille <= 3 mots)
# Cela nous permettra d'avoir des phrases générées plus cohérentes (et plus longues)

dataset_new_col_train = dataset_new_col_train.filter(lambda x: x["sentence_length"] > 3)
print("Nombre de phrases dans le train initial : ", dataset_train.num_rows)
print("Nombre de phrases dans le train filtré : ", dataset_new_col_train.num_rows)

# On fait pareil pour le dataset de test

dataset_new_col_test = (
    dataset["test"]
    .map(compute_sentence_length)
    .filter(lambda x: x["sentence_length"] > 3)
)

# ----------------------------- TRAIN/VALIDATION/TEST ---------------------------------------------

# On sépare maintenant notre datasein d'entrainement (train) en deux sets
# 1. Un d'entrainement pur (train) qui nous permettra d'apprendre sur la donnée fournie
# 2. Un de vérification (validation) sur lequel on évaluera notre modèle.
# Le dataset de validation nous donne un indice sur notre future performance
# en conditions réelles (données inconnues)

print("\n ----------- SPLIT DATASET ---------\n")

dataset_train_splitted = dataset_new_col_train.train_test_split(train_size=0.9, seed=42)
# Par défaut ce dataset s'appelle test, on le renomme par "validation"
dataset_train_splitted["validation"] = dataset_train_splitted.pop("test")

print(dataset_train_splitted)

# On le sauvegarde en local
dataset_train_splitted.save_to_disk("aurore/data/")

# # ---------------------------- PARTAGE SUR LE HUB DU DATASET -------------------------------------

# # HUGGING_FACE_PSEUDO = credentials["hugging_face_pseudo"]
# # HUGGING_FACE_DS_NAME = 'George_Sand'
# # dataset_train_splitted.push_to_hub(HUGGING_FACE_PSEUDO+"/"+ HUGGING_FACE_DS_NAME)
# # downloaded_dataset = load_dataset(HUGGING_FACE_PSEUDO+"/"+ HUGGING_FACE_DS_NAME)
# # print(downloaded_dataset)
